In [1]:
from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F,TChain,TGraph,TDatime,TLine,TLatex,TPad,TLegend,TPaveText,TF1,TGraphErrors,TGaxis
from ROOT import gROOT, gBenchmark, gRandom, gSystem, gPad,gStyle
from io import StringIO
import numpy as np
import ctypes
import os
from array import *
import math
import sys
from prettytable import PrettyTable
from decimal import Decimal, getcontext
import mpmath
import sympy as sp
import ezodf
from barion.amedata import AMEData
import matplotlib.pyplot as plt
from adjustText import adjust_text
from scipy.constants import *
from barion.particle import Particle
from barion.ring import Ring
from scipy.optimize import curve_fit
import requests
from bs4 import BeautifulSoup
import re
import textalloc as ta
import matplotlib.patches as patches

Welcome to JupyROOT 6.26/02


In [ ]:
me          = physical_constants['electron mass energy equivalent in MeV'][0]
amu         = physical_constants['atomic mass constant energy equivalent in MeV'][0]

In [ ]:
def gaussian(x, amplitude, mean, stddev):
    return amplitude * np.exp(-((x - mean) / stddev)**2 / 2)

In [ ]:
def Read_AME(file_path = "AME.rd"):
    try:
        # Try to open the file and read its contents
        with open(file_path, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        # If the file is not found, print an error message
        print(f"Error: File '{file_path}' not found.")
        return
    print(f"Reading mass data from file '{file_path}'...")
    # Initialize an empty list to store parsed data
    # Iterate through each line and parse the data
    data_list= []
    for line in lines:
        line = line.strip()  # Remove leading and trailing whitespace
        if line:  # Skip empty lines
            tokens = line.split()  # Split the line by spaces
            if len(tokens) == 6:  # Ensure each line has 6 fields
                entry = {
                    'Element': tokens[0],
                    'Z': int(tokens[1]),
                    'A': int(tokens[2]),
                    'ME (keV)': float(tokens[3]),
                    'ME_error (keV)': float(tokens[4]),
                    'Symbol': tokens[5] #this is not necessary
                }
                data_list.append(entry)
    return data_list

In [ ]:
def Read_AME_barion():
    ame = AMEData()  
    data_list= []
    for i in ame.ame_table:
        entry = {
            'Element': str(i[5])+i[6],
            'Z': int(i[4]),
            'A': int(i[5]),
            'ME (keV)': float(i[8]),
            'ME_error (keV)': float(i[9])
                }
        data_list.append(entry)
    return data_list

In [ ]:
def Read_elbien_file(file_path): # Previous method used
    data_list = []
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return data_list
    print(f"Reading electron binding energy data from file '{file_path}'...")
    header_line = None
    for line in lines:
        if line.startswith('#'):
            if header_line is None:
                header_line = line
            continue
        
        if not line.strip():
            continue  # Skip empty lines
        
        tokens = line.split()
        if header_line:
            headers = header_line.strip().split()[1:]  # Extract column headers
            entry = {'Z': int(tokens[0])}
            
            entry['TotBEn'] = int(tokens[1])
            # Add more fields up to '105e-ion'
            for i in range(len(headers), len(tokens)-2):
                entry[f'{i + 1}e-ion'] = int(tokens[i+2])
            data_list.append(entry)
    return data_list

In [ ]:
def Read_revlutiontime(file_path = "revlutiontime.txt"):
    try:
        # Try to open the file and read its contents
        with open(file_path, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        # If the file is not found, print an error message
        print(f"Error: File '{file_path}' not found.")
        return
    print(f"Reading revolution time from file '{file_path}'...")
    # Initialize an empty list to store parsed data
    # Iterate through each line and parse the data
    data_list = []
    for line in lines:
        line = line.strip()  # Remove leading and trailing whitespace
        if line.startswith('#'):
            continue
        if line:  # Skip empty lines
            tokens = line.split()  # Split the line by spaces
            entry = {
                'Element': tokens[0],
                'Z': int(float(tokens[1])),
                'Q': int(float(tokens[2])),
                'Flag': tokens[3],
                'RevolutionTime[ns]': float(tokens[4]),
                'Count': float(tokens[5]),
                'width of revolutionTime[ps]': float(tokens[6]),
                'error of width of revolutionTime[ps]': float(tokens[7])
            }
            data_list.append(entry)
    
    # Print the parsed data
    print("Total nuclei number: ",len(data_list))
    for entry in range (0, len(data_list)):
        print(data_list[entry])
    print("...")
        
    return data_list

In [ ]:
def GetAMEData(AME_data,element):    
    for entry_AME in AME_data:     
        if element == entry_AME['Element']:
            ME,MEError,A,N= entry_AME['ME (keV)']/1e3, entry_AME['ME_error (keV)']/1e3, entry_AME['A'], entry_AME['A'] - entry_AME['Z']
    return ME, MEError, A, N

In [ ]:
def GetBindingEnergy(elbien_data,Z,Q):
    match = next((entry for entry in elbien_data if entry['Z'] == Z), None)
    if match:
        key = f"{Q}e-ion"
        return match.get(key, 0) / 1e6
    return 0

In [2]:
def _retrieve_electron_binding_energies():
    # URL of the website
    url = "https://physics.nist.gov/cgi-bin/ASD/ie.pl"
    
    # Parameters for the POST request
    data = {
        'spectra': 'H-Og',
        'submit': 'Retrieve Data',
        'units': '1',
        'format': '0',
        'order': '0',
        'at_num_out': 'on',
        'ion_charge_out': 'on',
        'e_out': '1',
        'unc_out': 'on'
    }
    
    # Send a POST request to the website
    response = requests.post(url, data=data)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the table in the HTML
        table = soup.find('table', {'border': '1'})
        
        if table:
            # Initialize the dictionary
            data_dict = {}
    
            # Iterate over the rows of the table, skipping the header row
            for row in table.find_all('tr')[1:]:
                cells = row.find_all('td')
                if len(cells) == 4:
                    at_num = cells[0].text.strip()
                    ion_charge = cells[1].text.strip()
                    binding_energy = cells[2].text.strip()
                    uncertainty = cells[3].text.strip()
    
                    # Create a key-value pair in the dictionary
                    if at_num not in data_dict:
                        data_dict[at_num] = []
                    data_dict[at_num].append({
                        'Ion Charge': ion_charge,
                        'Binding Energy (eV)': binding_energy,
                        'Uncertainty (eV)': uncertainty
                    })
    
            # Print the dictionary
            for key, values in data_dict.items():
                print(f"At. Num. {key}:")
                for value in values:
                    print(f"  {value}")
        else: print("Table not found in the HTML content.")
    else: print("Failed to retrieve data from the website.")
    return data_dict

In [ ]:
def _save_electron_binding_energies(data_dict, filename = 'electron_binding_energies'):
    npz_data = {key: np.array(value) for key, value in data_dict.items()}
    # Save the data to a .npz file
    np.savez(filename, **npz_data)

In [ ]:
def _load_electron_binding_energies(filename = 'electron_binding_energies.npz'):
    data = np.load(filename, allow_pickle=True)
    # Convert the loaded data back to a dictionary format
    return {key: data[key].tolist() for key in data}

In [ ]:
def _convert_binding_energy_to_float(s):
    # Remove square brackets and parentheses
    s = s.strip('[]()')
    s = s.replace(' ', '')
    return float(s)

In [ ]:
def _get_ZQ_binding_energy_and_uncertainty(data_dict,Z,Q):
    ion = data_dict[f'{Z}'][Z-Q]
    binding_energy = _convert_binding_energy_to_float(ion['Binding Energy (eV)'])
    uncertainty = float(ion['Uncertainty (eV)'])
    return binding_energy, uncertainty

In [ ]:
def _get_electron_binding_energies(filename = 'electron_binding_energies.npz'):
    if os.path.exists(filename):
        return _load_electron_binding_energies(filename)
    else: 
        data_dict = _retrieve_electron_binding_energies()
        _save_electron_binding_energies(data_dict)
        return data_dict
    

In [ ]:
def _convert_binding_energy_to_float(s):
    # Remove square brackets and parentheses
    s = s.strip('[]()')
    s = s.replace(' ', '')
    return float(s)

In [ ]:
def LeastSquareFit(T, TError, MoQ, MoQError, p,iterationMax,A0,OutputMatrixOrNot,tol = None):
    N        =len(T)
    A        = A0.copy()
    chi2_min =1e20
    A_min    =A.copy()
    b2_inverse_min =[]
    chi2_previous = 1e20
    chi2 = 0 
    FDB = [[mpmath.power(f, k) for k in range(p)] for f in T]
    for iteration in range(iterationMax):
        delta_MoQ_fiDB= compute_delta_MoQ_fi(T, TError, A, p)
        PDB = [[1 / (MoQError[i] ** 2 + delta_MoQ_fiDB[i] ** 2) if i == j else 0 for j in range(N)] for i in range(N)]
        F = mpmath.matrix(FDB)
        P = mpmath.matrix(PDB)
        F_T = F.transpose()
        a1 = P *  mpmath.matrix(MoQ)
        a2 = F_T * a1
        b1 = P * F
        b2 = F_T * b1
        b2_sympy = sp.Matrix(b2.tolist())
        det_b2 = b2_sympy.det()
        
        if det_b2 == 0:
            raise ValueError("b2 = FT*P*F is a singular matrix. Iteration stopped.")
            
        b2_inverse = b2**-1
        A = b2_inverse * a2
        chi2 = compute_chi_squared(N, T, TError, MoQ, MoQError, A, p, delta_MoQ_fiDB)

        if chi2<chi2_min:
            chi2_min=chi2
            A_min   =A.copy() 
            if OutputMatrixOrNot: print("iteration  = ",iteration," A_min = ",A_min," chi2_min=", chi2_min)
                
        delta_chi2 = abs(chi2 - chi2_previous)
        if tol and delta_chi2 < tol:
                break
        chi2_previous = chi2
        
        if OutputMatrixOrNot: print_output(iteration, A, p, N, MoQ, MoQError, delta_MoQ_fiDB, T, TError, F, F_T, PDB, a1, a2, b1, b2, b2_inverse, chi2,A_min, chi2_min)
        print(f"Iteration = {iteration}, A_min = {A_min}, chi2_min = {np.round(float(chi2_min))}, delta_chi2 = {np.round(float(delta_chi2))}")
    return A_min, chi2_min,b2_inverse

In [ ]:
def compute_delta_MoQ_fi(T, TError, A, p):
    delta_MoQ_fiDB = []
    for i, (fi, delta_fi) in enumerate(zip(T, TError)):
        delta_MoQ_fi = sum(k * A[k] * mpmath.power(fi, k - 1) * delta_fi for k in range(1, p))
        delta_MoQ_fiDB.append(delta_MoQ_fi)
    return delta_MoQ_fiDB

In [ ]:
def compute_chi_squared(N, T, TError, MoQ, MoQError, A, p, delta_MoQ_fiDB):
    chi2 = 0
    for i in range(N):
        yfit = sum(A[k] * mpmath.power(T[i], k) for k in range(p))
        chi2 += (MoQ[i] - yfit) ** 2 / (MoQError[i] ** 2 + delta_MoQ_fiDB[i] ** 2)
    return chi2

In [ ]:
def print_output(iteration, A, p, N, MoQ, MoQError, delta_MoQ_fiDB, T, TError, F, F_T, PDB, a1, a2, b1, b2, b2_inverse, chi2, A_min, chi2_min):
    print(f"Iteration {iteration} - Best Fit Parameters:")
    for k in range(p):
        print(f"A[{k}] = {float(A_min[k]):.5e}")

    print("\nData and Uncertainties:")
    print("{:<15} {:<15} {:<15} {:<15}".format("MoQ", "MoQError", "delta_MoQ_fi", "T +/- TError"))
    for i in range(N):
        moq_val = float(MoQ[i])
        moq_err = float(MoQError[i])
        delta_moq_fi = float(delta_MoQ_fiDB[i])
        t_val = float(T[i])
        t_err = float(TError[i])

        print("{:<15.5f} {:<15.5e} {:<15.5e} {:<15.5f} +/- {:<15.5f}".format(moq_val, moq_err, delta_moq_fi, t_val, t_err))

    print("\nMatrices:")
    print("F:")
    for i in range(N):
        print("\t".join(f"{float(F[i, k]):.3f}" for k in range(p)))

    print("F_T:")
    for i in range(p):
        print("\t".join(f"{float(F_T[i, k]):.3e}" for k in range(N)))

    print("P:")
    for i in range(N):
        for j in range(N):
            pdb_val = float(PDB[i][j])
            if i == j:
                print("{:<6.3f}".format(pdb_val), end="\t")
            else:
                print("{:<1}".format(pdb_val), end="\t")
        print()

    print("\nIntermediate Results:")
    print("a1 = P * MoQ:")
    for i in range(N):
        print(f"{float(a1[i]):.5f}")

    print("a2 = F_T * P * MoQ:")
    for i in range(p):
        print(f"{float(a2[i]):.5f}")

    print("b1 = P * F:")
    for i in range(N):
        print("\t".join(f"{float(b1[i, k]):.3f}" for k in range(p)))

    print("b2 = F_T * P * F:")
    for i in range(p):
        print("\t".join(f"{float(b2[i, k]):.3f}" for k in range(p)))

    print("b2_inverse = (F_T * P * F)^-1:")
    for i in range(p):
        print("\t".join(f"{float(b2_inverse[i, k]):.3e}" for k in range(p)))

    print("\nFinal Chi-Squared:")
    print(f"Chi-squared: {float(chi2):.5f} (Minimum: {float(chi2_min):.5f})")


In [ ]:
def MassCalibration(Nuclei_Y,Nuclei_N,OutputMatrixOrNot_Y,p,A0_Y,iterationMax,tol = 0.1):
    Y_Y = [float(nucleus[14]) for nucleus in Nuclei_Y]
    YError_Y = [float(nucleus[15]) for nucleus in Nuclei_Y]
    X_Y = [float(nucleus[6]) for nucleus in Nuclei_Y]
    XError_Y = [float(nucleus[7]) for nucleus in Nuclei_Y]
    
    #LeastSquareFit
    A_min_Y, chi2_min_Y,b2_inverse_min_Y = LeastSquareFit (X_Y, XError_Y, Y_Y, YError_Y, p,iterationMax,A0_Y, OutputMatrixOrNot_Y,tol)
    
    table_list = []
    for nucleus in Nuclei_N:
        f = float(nucleus[6])
        delta_f = float(nucleus[7])

        # Calculate MoQ_N 
        MoQ_N = sum(A_min_Y[k] * mpmath.power(f, k) for k in range(p))

        # Calculate MoQ_N_fit_av 
        MoQ_N_fit_av = sum(b2_inverse_min_Y[k, l] * mpmath.power(f, k + l) for k in range(p) for l in range(p))
        MoQ_N_fit_av = mpmath.sqrt(MoQ_N_fit_av)

        MoQ_N_fit_sca = mpmath.sqrt(chi2_min_Y / (len(Y_Y) - p)) * MoQ_N_fit_av
        MoQ_N_fit_error = max(MoQ_N_fit_av, MoQ_N_fit_sca)  # 67th page of Matos's thesis
        #MoQ_N_fit_error= MoQ_N_fit_av  # this formular is used the old fortran code.
        MoQ_N_fre_error = sum(k * A_min_Y[k] * mpmath.power(f, k - 1) * delta_f for k in range(1, p))
        MoQ_N_tot_error = float(mpmath.sqrt(MoQ_N_fit_error**2 + MoQ_N_fre_error**2))   
        
        MoQ_N = float(MoQ_N)
        element          = nucleus[0]
        Z_N              = int(nucleus[1])
        A_N              = int(nucleus[2])
        N_N              = int(nucleus[3])
        Q_N              = float(nucleus[4])
        Flag             = nucleus[5]
        ME_N_AME         = float(nucleus[11]) # MeV
        MEError_N_AME    = float(nucleus[12]) # MeV
        EBindingEnergy_N = float(nucleus[13]) # MeV
        MoQ_N_AME        = float(nucleus[14]) # u/e
        MoQError_N_AME   = float(nucleus[15]) # u/e
        Mass_N           = MoQ_N * Q_N * amu 
        ME_N             = Mass_N - A_N*amu + Q_N*me - EBindingEnergy_N#EBindingEnergy # MeV *********************
        MEError_N        = MoQ_N_tot_error/MoQ_N * Mass_N # MeV
        table_list.append([element, Flag,Z_N, A_N,Q_N, f/1000, (ME_N*1000-ME_N_AME*1000), (ME_N*1000), (MEError_N*1000), (MEError_N_AME*1000)])
    return table_list,A_min_Y,chi2_min_Y

In [ ]:
def plot_residual_distribution(by,bye, bins = 15):
    plt.style.use('/home/duskdawn/analysis/plt-style.mplstyle')
    plt.rc('axes', unicode_minus=False)
    
    fig, axs = plt.subplots(1, 1, figsize=(25, 12))
    hist, bin_edges = np.histogram(by, bins=bins, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    initial_params = [110.0, 15.0, 12.0]  # Initial parameter guesses: amplitude, mean, stddev
    fit_params, _ = curve_fit(gaussian, bin_centers, hist, p0=initial_params)
    x_bins_fit = np.arange(bin_centers.min()*10000,bin_centers.max()*10000,1000)/10000
    
    xerr_bars = []
    for i in range(len(bin_centers)):
        bin_mask = (by >= bin_edges[i]) & (by <= bin_edges[i + 1]) if i == len(bin_centers) - 1 else (by >= bin_edges[i]) & (by < bin_edges[i + 1])
        bin_y_error = bye[bin_mask]
        xerr_bars.append(np.mean(bin_y_error) if len(bin_y_error) > 0 else 0)
    
    width = (bin_edges[1] - bin_edges[0]) - 0.2
    # Shade areas for 1, 2, and 3 sigma
    sigma_values = [fit_params[2] * i for i in range(1, 4)]
    colors = ['blue', 'green', 'yellow']

    for sigma, color in zip(sigma_values, colors):
        rect = patches.Rectangle((fit_params[1] - sigma, 0), 2 * sigma, 1, linewidth=1, edgecolor=color, facecolor=color, alpha=0.3)
        axs.add_patch(rect)
    axs.bar(bin_centers,  hist/ hist.max(), width=width, color='r', xerr=xerr_bars,
           edgecolor='red', linewidth=2 )
    axs.errorbar(bin_centers,hist / hist.max(),xerr=xerr_bars,fmt='bo',ecolor='black',elinewidth=5,capsize=4,alpha=0.3,markersize='13')
    axs.plot(x_bins_fit, gaussian(x_bins_fit, *fit_params)/ hist.max(), 
             'b-', label=rf'$\mu$ = {np.round(fit_params[1])}, $\sigma$ = {np.round(fit_params[2])}', linewidth = 3)
    
    
        
    axs.set_title('Residuals distribution',  fontsize=38)
    axs.set_xlabel(r"$\Delta_{exp} - \Delta_{AME}$ (keV)", fontsize=34)
    axs.set_ylabel('Normalized amplitude', fontsize=34)
    axs.tick_params(axis='both', which='major', labelsize=34)
    axs.legend()
    plt.show()

In [ ]:
def convert_name(name):
    import re
    # Use regex to extract the atomic number, isotope name, and charge
    atomic_num, element, charge = re.findall(r"\d+|\D+", name)
    # Format the atomic number and charge as superscripts
    atomic_num = '$^{'+str(atomic_num)+'}$'
    charge = '$^{'+str(charge)+'+}$'

    # Form the element symbol by capitalizing the first letter of the isotope name
    element = element.capitalize()

    # Combine the superscripts and element symbol
    return atomic_num + element + charge

In [ ]:
def strip_name(name):

    atomic_num, element, charge = re.findall(r"\d+|\D+", name)
    return atomic_num, element, charge

In [ ]:
def define_precision(p):
    if p ==3:
        precision = 20
    elif p ==4:
        precision = 31
    elif p==5:
        precision = 42
    elif p ==3:
        precision = 18
    elif p == 6:
        precision = 54
    elif p == 7:
        precision = 65
    else:
        precision = None
    return precision

In [1]:
def initial_seeds(p,x_data,y_data,y_data_err):
    if p ==7:
        def pol7(x,a,b,c,d,e,f,g):
            return a + b*x*1e3 + c*x**2*1e6+ d*x**3*1e9+ e*x**4*1e12 + f*x**5*1e15+g*x**6*1e18
        initial_guess = [-3.05960e+00,1.20092e-05 ,-3.41612e-12,-1.41612e-18, -5.41612e-24, 5e-32,-5e-37]
        params, covariance = curve_fit(pol7, x_data, y_data, p0=initial_guess, sigma = y_data_err, absolute_sigma = True)
    if p ==6:
        def pol6(x,a,b,c,d,e,f):
            return a + b*x*1e3 + c*x**2*1e6+ d*x**3*1e9+ e*x**4*1e12 + f*x**5*1e15
        initial_guess = [-3.05960e+00,1.20092e-05 ,-3.41612e-12,-1.41612e-18, -5.41612e-24, 5e-32]
        params, covariance = curve_fit(pol6, x_data, y_data, p0=initial_guess, sigma = y_data_err, absolute_sigma = True)
    if p ==5:
        def pol5(x,a,b,c,d,e):
            return a + b*x*1e3 + c*x**2*1e6+ d*x**3*1e9+ e*x**4*1e12
        initial_guess = [-3.05960e+00,1.20092e-05 ,-3.41612e-12,-1.41612e-18, -5.41612e-24]
        params, covariance = curve_fit(pol5, x_data, y_data, p0=initial_guess, sigma = y_data_err, absolute_sigma = True)
    elif p==4:
        def pol4(x,a,b,c,d):
            return a + b*x*1e3 + c*x**2*1e6+ d*x**3*1e9
        initial_guess = [-3.05960e+00,1.20092e-05 ,-3.41612e-12,-3.41612e-18]
        params, covariance = curve_fit(pol4, x_data, y_data, p0=initial_guess, sigma = y_data_err, absolute_sigma = True)
    elif p==3:
        def pol3(x,a,b,c):
            return a + b*x*1e3 + c*x**2*1e6
        initial_guess = [-3.05960e+00,1.20092e-05 ,-3.41612e-12]
        params, covariance = curve_fit(pol3, x_data, y_data, p0=initial_guess, sigma = y_data_err, absolute_sigma = True)
    elif p==2:
        def pol2(x,a,b):
            return a + b*x*1e3
        initial_guess = [0,1]
        params, covariance = curve_fit(pol2, x_data, y_data, p0=initial_guess, sigma = y_data_err, absolute_sigma = True)
    elif p==1:
        def pol1(x,a):
            return a
        initial_guess = [0]
        params, covariance = curve_fit(pol1, x_data, y_data, p0=initial_guess, sigma = y_data_err, absolute_sigma = True)
    return params, covariance

In [ ]:
def initial_seeds_root(p,x_data,y_data):
    if p ==6:
        func_Title = "[0] + [1]*x*1e3 + [2]*x*x*1e6+ [3]*x*x*x*1e9+ [4]*x*x*x*x*1e12+[5]*x*x*x*x*x*1e15"
        func = TF1("func",func_Title,0,700)
        func.SetParameters(-3.05960e+00,1.20092e-05 ,-3.41612e-12,-1.41612e-18, -5.41612e-24, 5e-32)
    
    if p ==5:
        func_Title = "[0] + [1]*x*1e3 + [2]*x*x*1e6+ [3]*x*x*x*1e9+ [4]*x*x*x*x*1e12"
        func = TF1("func",func_Title,0,700)
        func.SetParameters(-3.05960e+00,1.20092e-05 ,-3.41612e-12,-1.41612e-18, -5.41612e-24)
    
    elif p==4:
        func_Title = "[0] + [1]*x*1e3 + [2]*x*x*1e6+ [3]*x*x*x*1e9"
        func = TF1("func",func_Title,0,700)
        func.SetParameters(-3.05960e+00,1.20092e-05 ,-3.41612e-12,-3.41612e-18)
    elif p==3:
        func_Title = "[0] + [1]*x*1e3 + [2]*x*x*1e6"
        func = TF1("func",func_Title,0,700)
        func.SetParameters(-3.05960e+00,1.20092e-05 ,-5.41612e-12)
    
    elif p==2:
        func_Title = "[0] + [1]*x*1e3"
        func = TF1("func",func_Title,0,700)
    
    elif p==1:
        func_Title = "[0]"
        func = TF1("func",func_Title,0,700)
        
    gT_MoQ_Y = TGraphErrors()
    gT_MoQ_Y.SetName("gT_MoQ_Y")
    gT_MoQ_Y.SetTitle("The dependance of moq on T for calibrant nuclei")
    for i_Y in len(x_data):
        gT_MoQ_Y.SetPoint(i_Y,x_data,y_data)
        gT_MoQ_Y.SetPointError(i_Y,TError,Mass_QError)
    func.Print()
    gT_MoQ_Y.Fit(func,"RN")
    A0_Y=[]
    for i in range(0,p):
        A0_Y.insert(i, func.GetParameter(i))

In [ ]:
# Subplot 2: Residual Plot
def residual_plot(T_y,T_n,Nuclei_Y,Nuclei_N,A_mins_y,A_mins_n):
    plt.style.use('/home/duskdawn/analysis/plt-style.mplstyle')
    plt.rc('axes', unicode_minus=False)
    fig, axs = plt.subplots(1, 1, figsize=(25, 15))
    
    moq_y = [float(i) for i in np.array(Nuclei_Y)[:,14]]
    moq_ye = [float(i) for i in np.array(Nuclei_Y)[:,15]]
    if len(Nuclei_N) > 0:
        moq_n =  [float(i) for i in np.array(Nuclei_N)[:,14]]
        moq_ne = [float(i) for i in np.array(Nuclei_N)[:,15]]

    y_residul_vals_Y, y_residul_errs_Y, y_residul_vals_N, y_residul_errs_N = [[] for _ in range(4)]
    for i in range(len(Nuclei_Y)):
        x = T_y[i]
        yfit = 0
        for k in range(0,p):
            yfit = yfit + A_mins_y[i][k]*(x*1000)**k
        y_residul_vals_Y.append(float(yfit - moq_y[i]))
        y_residul_errs_Y.append(float(moq_ye[i]))
    for i in range(len(Nuclei_N)):
        x = T_n[i]
        yfit = 0
        for k in range(0,p):
            yfit = yfit + A_mins_n[i][k]*(x*1000)**k    
        y_residul_vals_N.append(float(yfit - moq_n[i]))
        y_residul_errs_N.append(float(moq_ne[i]))
    
 
    axs.errorbar(T_y, np.array(y_residul_vals_Y)*1e6, yerr=np.array(y_residul_errs_Y)*1e6, fmt='o', label="Calibrant ion",
                    markersize='13', ecolor='black',capsize=4, elinewidth=3)
    if len(T_n) > 0:
        axs.errorbar(T_n, np.array(y_residul_vals_N)*1e6, yerr=np.array(y_residul_errs_N)*1e6, fmt='o', label="Unknown ion",
                        markersize='13', ecolor='black',capsize=4, elinewidth=3)
    axs.axhline(y=0, color='red', linestyle='--')
    axs.set_xlabel(r"Revolution time, $T$ (ns)")
    axs.set_ylabel(r"$\left(m/q\right)_{fit} - \left(m/q\right)_{AME}$ ($\mu$u/e)")
    axs.set_title("Fit Residuals")
    axs.grid(True, linestyle=':', color='grey',alpha = 0.7)
    axs.legend(shadow=True,fancybox=True)
    return axs

In [ ]:
def iso_curve(revt, gammat, dp_p, sys, path = 108.36):
    return np.sqrt((((1-(path/(revt*(c/1e9)))**2-1/(gammat**2))*dp_p*revt)**2+sys**2))

In [ ]:
def calculate_reduced_chi_squared(y_data, y_fit, yerror, num_params):
    # Calculate the residuals
    residuals = y_data - y_fit
    # Calculate the chi-squared
    chi_squared = np.sum(residuals**2/yerror**2)
    # Calculate the degrees of freedom (dof)
    dof = len(y_data) - num_params
    # Calculate the reduced chi-squared
    chi_squared_red = chi_squared / dof
    return chi_squared_red

In [ ]:
def isochronicity_curve_plot(T,T_y,T_n,sT, Nuclei_Y,Nuclei_N,ref,names_latex, gammat_0 = 1.395, 
                             labels = False, fast = True, fts = 36):
    plt.style.use('/home/duskdawn/analysis/plt-style.mplstyle')
    plt.rc('axes', unicode_minus=False)
    fig, axs = plt.subplots(1, 1, figsize=(25, 15))
    
    y_sigmaT_vals_Y = np.array([float(i) for i in np.array(Nuclei_Y)[:,8]])
    y_sigmaT_errs_Y = np.array([float(i) for i in np.array(Nuclei_Y)[:,9]])
    y_sigmaT_errs_N = np.array([])
    sigma = np.concatenate((y_sigmaT_errs_Y, y_sigmaT_errs_N))
    x_fit = np.arange((T/1000).min()*1e6,(T/1000).max()*1e6, 10000) / 1e6
    axs.errorbar(T_y, y_sigmaT_vals_Y, yerr=y_sigmaT_errs_Y, fmt='o', label="Calibrant ion",
                    markersize='13', ecolor='black',capsize=4, elinewidth=3)
    if len(T_n) > 0:
        y_sigmaT_vals_N = [float(i) for i in np.array(Nuclei_N)[:,8]]
        y_sigmaT_errs_N = [float(i) for i in np.array(Nuclei_N)[:,9]]
        axs.errorbar(T_n, y_sigmaT_vals_N, yerr=y_sigmaT_errs_N, fmt='o', label="Unknown ion",
                        markersize='13', ecolor='black',capsize=4, elinewidth=3)
    
    sigma = np.concatenate((y_sigmaT_errs_Y, y_sigmaT_errs_N))
    seeds = [gammat_0, 0.0008, 0.01]
    fit_params, fit_covariance = curve_fit(iso_curve, T/1000, sT, p0=seeds,
                                      sigma = sigma, absolute_sigma = True)
    Gammat,         dpop,       sigma_t_sys  = fit_params
    GammatError, dpopError, sigma_t_sysError = np.sqrt(np.diag(fit_covariance))
    chi2_fit_sigma = calculate_reduced_chi_squared(sT, iso_curve(T/1000, *fit_params), sigma,len(fit_params))
    print('chi2_fit_sigma',chi2_fit_sigma)
    label_axs2 = f'$\sqrt{{\left(1 - \left(\\frac{{108.36}}{{T \cdot c}}\\right)^2 - \\frac{{1}}{{{Gammat:.4f}^2}}\\right)^2 \cdot \\left({np.abs(dpop)*1e-1:.4f} \cdot 10^{{-2}} \cdot T\\right)^2 + {abs(sigma_t_sys):.4f}^2}}$'

    axs.plot(x_fit, iso_curve(x_fit, *fit_params), 'r-', label=label_axs2,  linewidth=5)
    axs.set_xlabel(r"Revolution time, $T$ (ns)", fontsize = fts)
    axs.set_ylabel(r"$\sigma_{T}$ (ps)", fontsize = fts)

    #axs.set_title(f"Iso-curve, $\\gamma_t = {Gammat:.4f}$, $\\frac{{\\sigma_p}}{{p}} = {np.abs(dpop)*1e-1:.4f}$ $\%$, $\\sigma_{{sys}} = {abs(sigma_t_sys):.4f}$ ps", fontsize = fts)
    axs.grid(True, linestyle=':', color='grey',alpha = 0.7)
    if labels:
        index_with_ref = np.where(ref == 'Y')[0]
        index_without_ref = np.where(ref == 'N')[0]
        indexes_corrected = np.concatenate((index_with_ref,index_without_ref)).tolist()
        if fast:
            ta.allocate_text(fig,axs,T/1000,sT,names_latex[indexes_corrected],x_scatter=T/1000, y_scatter=sT,
                textsize=35, nbr_candidates=5000, seed = 42, min_distance=0.05, max_distance=0.4)
        else:
            latex_labels = []
            for i,index in enumerate(indexes_corrected):
                latex_labels.append(axs.annotate(names_latex[index], (T[i]/1000, sT[i]), fontsize=36))
            adjust_text(latex_labels, arrowprops=dict(arrowstyle='->', color='red'),alpha=.8)
            
    axs.legend(shadow=True,fancybox=True, fontsize = fts-5)
    axs.tick_params(axis='both', which='major', labelsize=fts)

    return axs

In [ ]:
def moq_plot(T_y,T_n,Nuclei_Y,Nuclei_N,A_mins_y,A_mins_n):
    # Subplot 1: MoQ Plot
    plt.style.use('/home/duskdawn/analysis/plt-style.mplstyle')
    plt.rc('axes', unicode_minus=False)
    
    fig, axs = plt.subplots(1, 1, figsize=(25, 15))

    T_vals = np.concatenate((T_y,T_n))
    moq_y = [float(i) for i in np.array(Nuclei_Y)[:,14]]
    moq_ye = [float(i) for i in np.array(Nuclei_Y)[:,15]]
    if len(Nuclei_N) > 0:
        moq_n =  [float(i) for i in np.array(Nuclei_N)[:,14]]
        moq_ne = [float(i) for i in np.array(Nuclei_N)[:,14]]
        A_mins = np.concatenate((A_mins_y,A_mins_n))
    else: 
        moq_n = []
        moq_ne = []
        A_mins = A_mins_y
    
    moq_vals = np.concatenate((moq_y,moq_n))
    moq_errs = np.concatenate((moq_ye,moq_ne))
    moq_fit = np.zeros_like(T_vals)
    for i,x in enumerate(T_vals):
        yfit = 0
        for k in range(p):
            yfit += A_mins[i][k] * (x * 1000) ** k
        moq_fit[i] = yfit
        
    axs.errorbar(T_vals, moq_vals, yerr=moq_errs, fmt='o', 
                    markersize='10', ecolor='black',capsize=4, elinewidth=3, label = f'AME 2020')
    
    text_params = []
    for k in range(0,p):
        coefficient = float(A_mins[0][k])
        exponent = int(mpmath.log10(abs(coefficient)))
        mantissa = coefficient / 10 ** exponent
        param = f"$a_{{{k}}} =$"
        if coefficient > 0: param = param + f' $+$ '
        param = param + f"${mantissa:.1f} \\cdot 10^{{{exponent}}}$"
        text_params.append(param)
    text_params_box_info = "\n".join(text_params)
    
    pol_fit_formula = []
    for k in range(0,p):
        term = f"$a_{{{k}}} \\cdot T^{{{k}}}$"
        pol_fit_formula.append(term)
    fit_label_pol = "+".join(pol_fit_formula)
    
    axs.plot(T_vals, moq_fit, 'r-', label=fit_label_pol, linewidth = 3)
    axs.set_xlabel(r"Revolution time, $T$ (ns)")
    axs.set_ylabel(r"$\left(m/q\right)_{AME20}$ (u/e)")
    axs.set_title("Mass-to-charge")
    axs.grid(True, linestyle=':', color='grey',alpha = 0.7)
    axs.legend(shadow=True,fancybox=True)
    axs.text(0.7, 0.2, text_params_box_info, transform=axs.transAxes, fontsize=25, 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    return axs

In [ ]:
def mass_excess_plot(ME_Exp_AME_y,ME_Exp_AME_n, MEError_Exp_y,MEError_Exp_n, 
                     MEError_AME_y,MEError_AME_n, MEError_sys_y,T_y,T_n, MEError_sys_n = [],fast = True, fts = 36):
    plt.style.use('/home/duskdawn/analysis/plt-style.mplstyle')
    plt.rc('axes', unicode_minus=False)
    
    fig, axs = plt.subplots(1, 1, figsize=(25, 15))
    MEError_sys_n = [MEError_sys_y[0] for i in range(len(MEError_AME_n))]
    ME_Exp_AME_T = np.concatenate((ME_Exp_AME_y,ME_Exp_AME_n))
    exp_errors = np.concatenate((MEError_Exp_y,MEError_Exp_n))
    ame_errors = np.concatenate((MEError_AME_y,MEError_AME_n))
    systematic_errors = np.concatenate((MEError_sys_y,MEError_sys_n))
    statistical_errors = [np.sqrt(exp**2+ame**2) for exp,ame in zip(exp_errors,ame_errors)]
    indexx = len(MEError_Exp_n)
    
    if indexx > 0:
        statistical_errors[-indexx:] = exp_errors[-indexx:]
        
    T_T = np.concatenate((T_y,T_n))
    total_errors = [np.sqrt(l**2+m**2) for l,m in zip(statistical_errors,systematic_errors)]
    systematic_errors_plot = [np.sqrt(total_errors**2-statistical_errors**2)+statistical_errors for total_errors,statistical_errors in zip(total_errors,statistical_errors)]

    # Set different colors for the error bars
    #axs.errorbar(T_T, ME_Exp_AME_T, yerr=systematic_errors_plot, fmt='bo', markersize=5, capsize=4, label='Systematic')
    axs.errorbar(T_T, ME_Exp_AME_T, yerr=total_errors, fmt='bo', markersize=5, capsize=4, label='Total')
    axs.errorbar(T_T, ME_Exp_AME_T, yerr=statistical_errors, fmt='ro', markersize=5, capsize=4, label='Statistical')
    
    axs.axhline(y=0, color='red', linestyle='--')
    axs.set_xlabel(r"Revolution time, $T$ (ns)", fontsize = fts)
    axs.set_ylabel(r"$\Delta_{exp} - \Delta_{AME}$ (keV)", fontsize = fts)
    #axs.set_title("Mass excess difference")
    axs.grid(True, linestyle=':', color='grey',alpha = 0.7)
    
    
    index_with_ref = np.where(ref == 'Y')[0]
    index_without_ref = np.where(ref == 'N')[0]
    indexes_corrected = np.concatenate((index_with_ref,index_without_ref)).tolist()

    if fast:
        ta.allocate_text(fig,axs,T_T,ME_Exp_AME_T,names_latex[indexes_corrected],x_scatter=T_T, y_scatter=ME_Exp_AME_T,
                textsize=35, nbr_candidates=5000, seed = 42, min_distance=0.05, max_distance=0.4)
    else:
        latex_labels = []
        for i,index in enumerate(indexes_corrected):
            latex_labels.append(axs.annotate(names_latex[index], (T_T[i], ME_Exp_AME_T[i]), fontsize=36))
        adjust_text(latex_labels, arrowprops=dict(arrowstyle='->', color='blue'),alpha=.8)
        
    axs.legend(shadow=True,fancybox=True, fontsize = fts-5)
    axs.tick_params(axis='both', which='major', labelsize=fts)
    return axs

In [10]:
def cmm_subplots(T_y,T_n,Nuclei_Y,Nuclei_N,A_mins_y,A_mins_n,T,sT,ref,names_latex,ME_Exp_AME_y,
                 ME_Exp_AME_n, MEError_Exp_y,MEError_Exp_n, MEError_AME_y,MEError_AME_n, 
                 MEError_sys_y,MEError_sys_n, gammat_0 = 1.395, fast = True):
    plt.style.use('/home/duskdawn/analysis/plt-style.mplstyle')
    plt.rc('axes', unicode_minus=False)
    
    fig, axs = plt.subplots(2, 2, figsize=(25, 15))
    axs = axs.ravel()
    y_sigmaT_vals_Y = np.array([float(i) for i in np.array(Nuclei_Y)[:,8]])
    y_sigmaT_errs_Y = np.array([float(i) for i in np.array(Nuclei_Y)[:,9]])
    y_sigmaT_errs_N = np.array([])
    sigma = np.concatenate((y_sigmaT_errs_Y, y_sigmaT_errs_N))
    #moq subplot 1
    T_vals = np.concatenate((T_y,T_n))
    moq_y = [float(i) for i in np.array(Nuclei_Y)[:,14]]
    moq_ye = [float(i) for i in np.array(Nuclei_Y)[:,15]]
    if len(Nuclei_N) > 0:
        moq_n =  [float(i) for i in np.array(Nuclei_N)[:,14]]
        moq_ne = [float(i) for i in np.array(Nuclei_N)[:,15]]
        A_mins = np.concatenate((A_mins_y,A_mins_n))
    else: 
        moq_n = []
        moq_ne = []
        A_mins = A_mins_y
    
    moq_vals = np.concatenate((moq_y,moq_n))
    moq_errs = np.concatenate((moq_ye,moq_ne))
    moq_fit = np.zeros_like(T_vals)
    for i,x in enumerate(T_vals):
        yfit = 0
        for k in range(p):
            yfit += A_mins[i][k] * (x * 1000) ** k
        moq_fit[i] = yfit
        
    axs[0].errorbar(T_vals, moq_vals, yerr=moq_errs, fmt='o', 
                    markersize='10', ecolor='black',capsize=4, elinewidth=3, label = f'AME 2020')
    
    text_params = []
    for k in range(0,p):
        coefficient = float(A_mins[0][k])
        exponent = int(mpmath.log10(abs(coefficient)))
        mantissa = coefficient / 10 ** exponent
        param = f"$a_{{{k}}} =$"
        if coefficient > 0: param = param + f' $+$ '
        param = param + f"${mantissa:.1f} \\cdot 10^{{{exponent}}}$"
        text_params.append(param)
    text_params_box_info = "\n".join(text_params)
    
    pol_fit_formula = []
    for k in range(0,p):
        term = f"$a_{{{k}}} \\cdot T^{{{k}}}$"
        pol_fit_formula.append(term)
    fit_label_pol = "+".join(pol_fit_formula)
    
    axs[0].plot(T_vals, moq_fit, 'r-', label=fit_label_pol, linewidth = 3)
    axs[0].set_xlabel(r"Revolution time, $T$ (ns)")
    axs[0].set_ylabel(r"$\left(m/q\right)_{AME20}$ (u/e)")
    axs[0].set_title("Mass-to-charge")
    axs[0].grid(True, linestyle=':', color='grey',alpha = 0.7)
    axs[0].legend(shadow=True,fancybox=True)
    axs[0].text(0.7, 0.2, text_params_box_info, transform=axs[0].transAxes, fontsize=25, 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    #Residual subplot 2
    moq_y = [float(i) for i in np.array(Nuclei_Y)[:,14]]
    moq_ye = [float(i) for i in np.array(Nuclei_Y)[:,15]]
    if len(Nuclei_N) > 0:
        moq_n =  [float(i) for i in np.array(Nuclei_N)[:,14]]
        moq_ne = [float(i) for i in np.array(Nuclei_N)[:,15]]

    y_residul_vals_Y, y_residul_errs_Y, y_residul_vals_N, y_residul_errs_N = [[] for _ in range(4)]
    for i in range(len(Nuclei_Y)):
        x = T_y[i]
        yfit = 0
        for k in range(0,p):
            yfit = yfit + A_mins_y[i][k]*(x*1000)**k
        y_residul_vals_Y.append(float(yfit - moq_y[i]))
        y_residul_errs_Y.append(float(moq_ye[i]))
    for i in range(len(Nuclei_N)):
        x = T_n[i]
        yfit = 0
        for k in range(0,p):
            yfit = yfit + A_mins_n[i][k]*(x*1000)**k    
        y_residul_vals_N.append(float(yfit - moq_n[i]))
        y_residul_errs_N.append(float(moq_ne[i]))
    axs[1].errorbar(T_y, np.array(y_residul_vals_Y)*1e6, yerr=np.array(y_residul_errs_Y)*1e6, fmt='o', label="Calibrant ion",
                    markersize='13', ecolor='black',capsize=4, elinewidth=3)
    if len(T_n) > 0:
        axs[1].errorbar(T_n, np.array(y_residul_vals_N)*1e6, yerr=np.array(y_residul_errs_N)*1e6, fmt='o', label="Unknown ion",
                        markersize='13', ecolor='black',capsize=4, elinewidth=3)
    axs[1].axhline(y=0, color='red', linestyle='--')
    axs[1].set_xlabel(r"Revolution time, $T$ (ns)")
    axs[1].set_ylabel(r"$\left(m/q\right)_{fit} - \left(m/q\right)_{AME}$ ($\mu$u/e)")
    axs[1].set_title("Fit Residuals")
    axs[1].grid(True, linestyle=':', color='grey',alpha = 0.7)
    axs[1].legend(shadow=True,fancybox=True)
    
    #iso curve subplot 3
    fit_params, fit_covariance = curve_fit(iso_curve, T/1000, sT, p0=[gammat_0, 0.0001, 0.001],
                                      sigma = sigma, absolute_sigma = True)
    Gammat,         dpop,       sigma_t_sys  = fit_params
    GammatError, dpopError, sigma_t_sysError = np.sqrt(np.diag(fit_covariance))
    chi2_fit_sigma = calculate_reduced_chi_squared(sT, iso_curve(T/1000, *fit_params), sigma,len(fit_params))
    print('chi2_fit_sigma',chi2_fit_sigma)

    y_sigmaT_vals_Y = [float(i) for i in np.array(Nuclei_Y)[:,8]]
    y_sigmaT_errs_Y = [float(i) for i in np.array(Nuclei_Y)[:,9]]
    
    
    x_fit = np.arange((T/1000).min()*1e6,(T/1000).max()*1e6, 10000) / 1e6
    axs[2].errorbar(T_y, y_sigmaT_vals_Y, yerr=y_sigmaT_errs_Y, fmt='o', label="Calibrant ion",
                    markersize='13', ecolor='black',capsize=4, elinewidth=3)
    if len(T_n) > 0:
        y_sigmaT_vals_N = [float(i) for i in np.array(Nuclei_N)[:,8]]
        y_sigmaT_errs_N = [float(i) for i in np.array(Nuclei_N)[:,9]]
        axs[2].errorbar(T_n, y_sigmaT_vals_N, yerr=y_sigmaT_errs_N, fmt='o', label="Unknown ion",
                        markersize='13', ecolor='black',capsize=4, elinewidth=3)
    label_axs = f'$\sqrt{{\left(1 - \left(\\frac{{108.36}}{{T \cdot c}}\\right)^2 - \\frac{{1}}{{\gamma_t^2}}\\right)^2 \cdot \\left(\\frac{{\sigma_{{p}}}}{{p}} \cdot T\\right)^2 + \sigma_{{sys}}^2}}$'
    
    axs[2].plot(x_fit, iso_curve(x_fit, *fit_params), 'r-', label=label_axs,  linewidth=5)
    axs[2].set_xlabel(r"Revolution time, $T$ (ns)")
    axs[2].set_ylabel(r"$\sigma_{T}$ (ps)")
    
    axs[2].set_title(f"Iso-curve, $\\gamma_t = {Gammat:.4f}$, $\\frac{{\\sigma_p}}{{p}} = {dpop*1e-1:.4f}$ $\%$, $\\sigma_{{sys}} = {abs(sigma_t_sys):.4f}$ ps")
    
    axs[2].grid(True, linestyle=':', color='grey',alpha = 0.7)
    #latex_labels = []
    index_with_ref = np.where(ref == 'Y')[0]
    index_without_ref = np.where(ref == 'N')[0]
    indexes_corrected = np.concatenate((index_with_ref,index_without_ref)).tolist()
    #for i,index in enumerate(indexes_corrected):
    #    latex_labels.append(axs[2].annotate(names_latex[index], (T[i]/1000, sT[i]), fontsize=36))
    #adjust_text(latex_labels, arrowprops=dict(arrowstyle='->', color='red'),alpha=.8)
    axs[2].legend(shadow=True,fancybox=True)
    
    #mass excess subplot 4
    ME_Exp_AME_T = np.concatenate((ME_Exp_AME_y,ME_Exp_AME_n))
    exp_errors = np.concatenate((MEError_Exp_y,MEError_Exp_n))
    ame_errors = np.concatenate((MEError_AME_y,MEError_AME_n))
    systematic_errors = np.concatenate((MEError_sys_y,MEError_sys_n))
    statistical_errors = [np.sqrt(exp**2+ame**2) for exp,ame in zip(exp_errors,ame_errors)]
    indexx = len(np.where(ref=='N'))
    if indexx > 0:
        statistical_errors[-indexx:] = exp_errors[-indexx:]
    T_T = np.concatenate((T_y,T_n))
    total_errors = [np.sqrt(l**2+m**2) for l,m in zip(statistical_errors,systematic_errors)]
    # Set different colors for the error bars
    axs[3].errorbar(T_T, ME_Exp_AME_T, yerr=total_errors, fmt='bo', markersize=5, capsize=4, label='Total')
    axs[3].errorbar(T_T, ME_Exp_AME_T, yerr=statistical_errors, fmt='ro', markersize=5, capsize=4, label='Statistical')
    
    axs[3].axhline(y=0, color='red', linestyle='--')
    axs[3].set_xlabel(r"Revolution time, $T$ (ns)")
    axs[3].set_ylabel(r"$\Delta_{exp} - \Delta_{AME}$ (keV)")
    axs[3].set_title("Mass excess difference")
    axs[3].grid(True, linestyle=':', color='grey',alpha = 0.7)
    
    
    if fast:
        ta.allocate_text(fig,axs[3],T_T,ME_Exp_AME_T,names_latex[indexes_corrected],x_scatter=T_T, y_scatter=ME_Exp_AME_T,
                textsize=35, nbr_candidates=5000, seed = 42, min_distance=0.05, max_distance=0.4)
    else:
        latex_labels = []
        for i,index in enumerate(indexes_corrected):
            latex_labels.append(axs[3].annotate(names_latex[index], (T_T[i], ME_Exp_AME_T[i]), fontsize=36))
        adjust_text(latex_labels, arrowprops=dict(arrowstyle='->', color='blue'),alpha=.8)
    axs[3].legend(shadow=True,fancybox=True)
    plt.tight_layout()
    plt.show()

In [ ]:
print("Welcome to the mass calibration code :) ")
print("I hope you get some exciting measurements! ")

In [ ]:
def process_nuclei(data, AME_data, elbien_data):
    '''
    Nucleus lists with
    0       1 2 3 4    5  6     7      8          9         10     11     12          13           14      15
    element,Z,A,N,Q, Flag,T, Terror, SigmaT, SigmaTError, latex,   ME, MEError, EBindingEnergy, Mass_Q, Mass_QError
    '''
    
    Nuclei, Nuclei_Y, Nuclei_N = [], [], []

    for ion in data:
        Z, Q = int(ion[1]), int(ion[2])  # Extract Z and Q once and reuse
        ME, MEError, A, N = GetAMEData(AME_data, ion[0])  # MeV
        EBindingEnergy, EBindingEnergy_uncertainty = _get_ZQ_binding_energy_and_uncertainty(elbien_data, Z, Q)
        
        # Convert to MeV and calculate Mass and Mass_Q
        EBindingEnergy /= 10**6  # MeV
        EBindingEnergy_uncertainty /= 10**6  # MeV
        Mass = A * amu + ME - Q * me + EBindingEnergy
        Mass_error = np.sqrt(MEError**2 + EBindingEnergy_uncertainty**2)
        Mass_Q = Mass / (amu * Q)
        Mass_QError = Mass_error / (amu * Q)

        # Build the complete nucleus data list
        nucleus = ion.tolist() + [ME, MEError, EBindingEnergy, Mass_Q, Mass_QError]
        nucleus.insert(2, A)
        nucleus.insert(3, N)
        Nuclei.append(nucleus)
        
        # Append to Nuclei_Y or Nuclei_N based on Flag
        if ion[3] == "Y": Nuclei_Y.append(nucleus)
        if ion[3] == "N": Nuclei_N.append(nucleus)

    # Reporting
    if Nuclei_N:
        print(f"Total nuclei number: {len(Nuclei)}, calibrant nuclei: {len(Nuclei_Y)}, unknown nuclei: {len(Nuclei_N)}, First unknown nucleus: {Nuclei_N[0][0]}")
    else:
        print(f"Total nuclei number: {len(Nuclei)}, calibrant nuclei: {len(Nuclei_Y)}, unknown nuclei: {len(Nuclei_N)}, None")
    
    return Nuclei, Nuclei_Y, Nuclei_N

In [ ]:
def get_initial_seeds(p, nuclei_data):

    # Ensure nuclei_data is a NumPy array
    nuclei_data = np.asarray(nuclei_data)

    # Extracting T, Mass_Q, and Mass_QError from nuclei_data
    T = nuclei_data[:, 6].astype(float) / 1000
    Mass_Q = nuclei_data[:, 14].astype(float)
    Mass_QError = nuclei_data[:, 15].astype(float)

    # Get initial parameter estimates and covariance matrix
    params, cov = initial_seeds(p, T, Mass_Q, Mass_QError)
    print(f'intiial fit errors = {np.sqrt(np.diag(cov))}')
    
    return params

In [ ]:
def self_calibration(Nuclei_Y, p, A0_Y, iterationMax):
    print("# 1. self-calibration.")
    # Goes over every reference ion and compute their m/q from the fit considering all others known but it
    table = PrettyTable()
    table.field_names = ["Ion", "Ref?", "TOF (ns)", "ME(EXP-AME)(keV)", "ME_EXP(keV)", 
                         "ERROR(EXP,keV)", "ERROR(AME,keV)"]
    
    # Initialize lists to store data
    chi_min_list, T_y, ME_Exp_AME_y, MEError_Exp_y, MEError_AME_y, A_mins_y,table_listDB = [], [], [], [], [], [], []
    
    # Iterate over each nucleus for self-calibration
    for i, nucleus_y in enumerate(Nuclei_Y):
        Nuclei_Y_SelfCal = [Nuclei_Y[j] for j in range(len(Nuclei_Y)) if i != j]
        Nuclei_N_SelfCal = [nucleus_y]
    
        table_list, A_min_Y, chi2_min_Y = MassCalibration(Nuclei_Y_SelfCal, Nuclei_N_SelfCal, False, p, A0_Y, iterationMax, tol=1e-10)
    
        # Extract and store calibration results
        A_mins_y.append(A_min_Y)
        calibration_result = table_list[0]
        table_listDB.append(calibration_result)
        
        # Adding row to the table
        ion_label = f"{calibration_result[0]}{int(calibration_result[4])}+"
        table.add_row([ion_label, calibration_result[1]] + ["%10.6f" % value for value in calibration_result[5:10]])
        
        # Append results to lists
        T_y.append(calibration_result[5])
        ME_Exp_AME_y.append(calibration_result[6])
        MEError_Exp_y.append(calibration_result[8])
        MEError_AME_y.append(calibration_result[9])
        chi_min_list.append(chi2_min_Y / (len(Nuclei_Y) - p))
        
        # Print statement for each nucleus
        print(f"unknown nucleus is {ion_label}, ME(exp-ame) = {calibration_result[6]:.3f} \u00B1 {calibration_result[8]:.3f} keV")
    
    # Print the entire table
    print(table)
    return chi_min_list, T_y, ME_Exp_AME_y, MEError_Exp_y, MEError_AME_y, A_mins_y,table_listDB,table

In [ ]:
def determination_of_systematic_error(table_listDB, ME_Exp_AME_y, MEError_Exp_y, MEError_AME_y, Nuclei_Y, p):
    
    MEError_Exp_array = np.array([entry[9] for entry in table_listDB])
    chi2_a = sum((me_exp_ame**2 / (me_error_exp**2 + me_error_ame**2)) 
                 for me_exp_ame, me_error_exp, me_error_ame in zip(ME_Exp_AME_y, MEError_Exp_y, MEError_AME_y))
    
    # Initial normalized chi-squared calculation
    chin_a = chi2_a / (len(Nuclei_Y) - 1 - p) # 1 for the one we consider as unknown in the self calibration
    
    # Systematic error adjustment loop
    MEError_sys, chin_b = 0,0
    if chin_a > 1:
        while True:
            chi2_b = sum((me_exp_ame**2 / (me_error_exp**2 + me_error_ame**2 + MEError_sys**2)) 
                         for me_exp_ame, me_error_exp, me_error_ame in zip(ME_Exp_AME_y, MEError_Exp_y, MEError_AME_y))
            chin_b = chi2_b / (len(table_listDB) - p -1)
    
            if chin_b <= 1:
                break
    
            MEError_sys += 0.01
    
    MEError_sys_y = [MEError_sys] * len(table_listDB)
    
    info_table = PrettyTable()

    # Define the table columns
    info_table.field_names = ["Description", "Value"]
    
    # Add rows to the table
    info_table.add_row(["Fitting function", f"m/q(T)=a0+a1*T+a2*T^2+...+an*T^n"])
    info_table.add_row(["Fit order n", p - 1])
    #info_table.add_row(["A_min_Y", A_mins_y])
    info_table.add_row(["Normalized χ²", chin_a])
    info_table.add_row(["Normalized χ² (+systematic error)", chin_b])
    info_table.add_row(["Systematic error", f"{MEError_sys} keV"])
    info_table.add_row(["REF_NUC NO", len(Nuclei_Y)])
    
    # Print the table
    print(info_table)
    return MEError_sys_y

In [ ]:
def process_sheet(sheet, exclusion_list):
    data = []
    # careful with the format of the ods. It must have 7 columns even if the 2nd is irrelevant, modify
    for row in range(sheet.nrows()):
        if row in exclusion_list:
            continue

        row_data = [sheet[row, col].value for col in range(sheet.ncols()) if sheet[row, col].value is not None]
        if row_data:
            # Ensure row_data has a consistent length
            row_data += [None] * (sheet.ncols() - len(row_data))
            data.append(row_data)

    # Convert to a 2D numpy array with dtype=object to handle mixed data types
    return np.array(data, dtype=object)

In [ ]:
def get_processed_data(sheet_data, unknown = None):
    names = sheet_data[:, 0]
    harmonics = sheet_data[:, 2].astype(int)
    f = sheet_data[:, 3].astype(float)
    fe = sheet_data[:, 4].astype(float)
    s = sheet_data[:, 5].astype(float)
    se = sheet_data[:, 6].astype(float)

    names_latex = [convert_name(name) for name in names]
    stripped = np.array([strip_name(name) for name in names])
    atomic_num, element, charge = stripped[:,0], stripped[:,1], stripped[:,2]
    name_aux = [str(a) + str(e) for a, e in zip(atomic_num, element)]

    T = 1e12 / (f / harmonics)
    sT = s / f * T
    eT = T * fe / f
    er = fe / (f**2) * harmonics * 1e12

    zz = []
    for ele in element:
        for i in AMEData().ame_table:
            if str(i[6]) == str(ele):
                zz.append(int(i[4]))
                break
    
    references = ['Y' for _ in charge]
    if unknown:
        for unknown_ion in unknown:
            index = np.where(names == unknown_ion)[0][0]
            references[index] = 'N'
    data = np.column_stack([name_aux, zz, charge, references, T, er, sT, eT,names_latex])
    return data

In [ ]:
def save_data_to_file(filename, data):
    with open(filename, 'w') as file:
        file.write('\n'.join(['\t'.join(map(str, row)) for row in data]) + '\n')

In [ ]:
def final_table(table, MEError_sys_y):
    fieldname = 'ERROR(SYS,kEV)'
    table.field_names.insert(8, fieldname) 
    table.align[fieldname] = 'c' 
    table.valign[fieldname] = 't' 
    for i, _ in enumerate(table.rows): 
        table.rows[i].insert(8, MEError_sys_y) 
    return table

In [ ]:
def calibration_of_unknown_nuclei(Nuclei_Y, Nuclei_N, p, A0_Y, iterationMax):
    print("# 2. mass calibration for nuclei with unknown mass.")
    table_u = PrettyTable()
    table_u.field_names = ["Ion", "Ref?", "TOF (ns)", "ME(EXP-AME)(keV)", "ME_EXP(keV)", 
                           "ERROR(EXP,keV)", "ERROR(AME,keV)"]
    
    if len(Nuclei_N) > 0:
        table_list, A_min_Y, chi2_min_Y = MassCalibration(Nuclei_Y, Nuclei_N, False, p, A0_Y, iterationMax, tol=1e-10)
        MEError_Exp_array = np.array([])
        T_n, ME_Exp_AME_n, MEError_Exp_n, MEError_AME_n = [], [], [], []
    
        for entry in table_list:
            ion_label = f"{entry[0]}{int(entry[4])}+"
            table_u.add_row([ion_label, entry[1]] + ["%10.6f" % value for value in entry[5:10]])
            MEError_Exp_array = np.append(MEError_Exp_array, entry[8])
            T_n.append(entry[5])
            ME_Exp_AME_n.append(entry[6])
            MEError_Exp_n.append(entry[8])
            MEError_AME_n.append(entry[9])
        
        print("Fitting function: m/q(T)=a0+a1*T+a1*T^2+...+a1*T^n")
        print("Fit order n:", p - 1)
        print("A_min_Y:", A_min_Y)
        print("normalized χ²:", chi2_min_Y / (len(Nuclei_Y) - p))
        print("REF_NUC NO:", len(Nuclei_Y))
    
        # Print the table
        print(table_u)
        return T_n, ME_Exp_AME_n, MEError_Exp_n, MEError_AME_n
    else:
        print("No nuclei with unknown mass for calibration.")
        return [], [], [], []

In [ ]:
def controller(filename, revname, sheet_index, exclusion_list, p, iterationMax, unknown = None):
    # Importing data
    odsinfo = ezodf.opendoc(filename)
    sheet = odsinfo.sheets[sheet_index]
    sheet_data = process_sheet(sheet, exclusion_list)
    processed_data = get_processed_data(sheet_data, unknown = unknown)
    
    save_data_to_file(revname, processed_data)
    precision = define_precision(p)
    mpmath.mp.dps = precision
    # Read external info
    AME_data            = Read_AME_barion()
    elbien_data = _get_electron_binding_energies()
    # AME values for each nuclei
    Nuclei, Nuclei_Y, Nuclei_N = process_nuclei(processed_data, AME_data, elbien_data)
    
    A0_Y = get_initial_seeds(p, Nuclei_Y)
    #self calibration
    chi_min_list, T_y, ME_Exp_AME_y, MEError_Exp_y, MEError_AME_y, A_mins_y,table_listDB,table = self_calibration(Nuclei_Y, p, A0_Y, iterationMax)
    MEError_sys_y = determination_of_systematic_error(table_listDB, ME_Exp_AME_y, MEError_Exp_y, MEError_AME_y, Nuclei_Y, p)
    finaltable = final_table(table, MEError_sys_y[0])
    #mass calibration unknown nuclei
    T_n, ME_Exp_AME_n, MEError_Exp_n, MEError_AME_n = calibration_of_unknown_nuclei(Nuclei_Y, Nuclei_N, p, A0_Y, iterationMax)
    
    return processed_data,T_n, ME_Exp_AME_n, MEError_Exp_n, MEError_AME_n, MEError_sys_y, chi_min_list, T_y, ME_Exp_AME_y, MEError_Exp_y, MEError_AME_y, A_mins_y,table_listDB,table, Nuclei, Nuclei_Y, Nuclei_N